In [75]:
# --- Basic Imports ---
from utils import *

# Reading Data

## Normalizing columns and information

In [76]:
df = pd.read_csv(os.path.join(BRONZE_PATH, 'bank-additional-full.csv'), sep=';')
df.columns = [normalizeString(colname) for colname in df.columns]

In [77]:
print("Dataframe shape:", df.shape)
pprint(("Columns:", df.columns.tolist()), compact=True)
print((df.info()))
print(df.shape)

Dataframe shape: (41188, 21)
('Columns:',
 ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
  'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous',
  'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m',
  'nr_employed', 'y'])
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campai

## Check for inconsistent categorical variables

In [78]:
cat_cols = df.select_dtypes(include=["object", "category"]).columns
max_values = 20

print(f"Found {len(cat_cols)} categorical columns.\n")

for col in cat_cols:
    uniques = df[col].unique()
    n_unique = len(uniques)
    
    print(f"--- {col} ---")
    print(f"Unique values ({n_unique}):")
    
    if n_unique <= max_values:
        pprint(sorted(uniques))
    else:
        pprint(sorted(uniques[:max_values]))
        print(f"... and {n_unique - max_values} more")
    
    print("\n")

Found 11 categorical columns.

--- job ---
Unique values (12):
['admin.',
 'blue-collar',
 'entrepreneur',
 'housemaid',
 'management',
 'retired',
 'self-employed',
 'services',
 'student',
 'technician',
 'unemployed',
 'unknown']


--- marital ---
Unique values (4):
['divorced', 'married', 'single', 'unknown']


--- education ---
Unique values (8):
['basic.4y',
 'basic.6y',
 'basic.9y',
 'high.school',
 'illiterate',
 'professional.course',
 'university.degree',
 'unknown']


--- default ---
Unique values (3):
['no', 'unknown', 'yes']


--- housing ---
Unique values (3):
['no', 'unknown', 'yes']


--- loan ---
Unique values (3):
['no', 'unknown', 'yes']


--- contact ---
Unique values (2):
['cellular', 'telephone']


--- month ---
Unique values (10):
['apr', 'aug', 'dec', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep']


--- day_of_week ---
Unique values (5):
['fri', 'mon', 'thu', 'tue', 'wed']


--- poutcome ---
Unique values (3):
['failure', 'nonexistent', 'success']


--- y ---


No inconsistent categories - OK

## Drop duplicates

In [79]:
print(f'Ammount of duplicate rows: {df.duplicated(keep=False).sum()}')
print(f'Total rows: {len(df)}')

Ammount of duplicate rows: 24
Total rows: 41188


In [80]:
df = df.drop_duplicates()
print(f'Total rows after dropping duplicates: {len(df)}')

Total rows after dropping duplicates: 41176


## Categorical values normalization/mapping
From EDA we can see that:
- Target variable is categorical;
- pdays is very skewed making it "categorical"
- day_of_week - shouldn't be converted with one-hot to keep order
- month - shouldn't be converted with one-hot to keep order

In [81]:
# print unique values of pdays and respective ammount of unique values
df['pdays'].groupby(df['pdays']).size().sort_values(ascending=False)

pdays
999    39661
3        439
6        412
4        118
9         64
2         61
7         60
12        58
10        52
5         46
13        36
11        28
1         26
15        24
14        20
8         18
0         15
16        11
17         8
18         7
19         3
22         3
21         2
20         1
25         1
26         1
27         1
Name: pdays, dtype: int64

In [82]:
month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
print(df['month'].unique())

dow_map = {
    'mon': 1, 'tue': 2, 'wed': 3, 'thu': 4, 'fri': 5
}
print(df['day_of_week'].unique())

# Convert target column
df = convert_target(df, target_col='y')

# Convert pdays into 0 or 1
df['pdays'] = df['pdays'].apply(lambda x: 0 if x == 999 else 1)

# Map month and day_of_week to numerical values for easier analysis and modeling
df['month'] = df['month'].map(month_map)
df['day_of_week'] = df['day_of_week'].map(dow_map)

['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
['mon' 'tue' 'wed' 'thu' 'fri']


## Drop columns that introduce leakage

In [83]:
df = df.drop(columns=['duration'])

# Save base dataset without feature engineering

In [84]:
print("Dataframe shape:", df.shape)
pprint(("Columns:", df.columns.tolist()), compact=True)
print((df.info()))
print(df.shape)

Dataframe shape: (41176, 20)
('Columns:',
 ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
  'month', 'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
  'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed',
  'y'])
<class 'pandas.core.frame.DataFrame'>
Index: 41176 entries, 0 to 41187
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41176 non-null  int64  
 1   job             41176 non-null  object 
 2   marital         41176 non-null  object 
 3   education       41176 non-null  object 
 4   default         41176 non-null  object 
 5   housing         41176 non-null  object 
 6   loan            41176 non-null  object 
 7   contact         41176 non-null  object 
 8   month           41176 non-null  int64  
 9   day_of_week     41176 non-null  int64  
 10  campaign        41176 non-null  int64  
 11  pdays           41176 n

In [85]:
df.to_csv(os.path.join(SILVER_PATH, 'base_dataset.csv'), sep=";", index=False)